Teste Prático para Engenheiros de Dados - Itaú RTDA

In [140]:
# Importa libs necessárias

# Ex.: 01
import pandas as pd
import boto3
import io

# Ex.: 02
from pyspark.sql import SparkSession, functions as f
from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType

1. ETL e Manipulação de Dados

Utilize o arquivo sales_data.csv e:
Limpe os dados removendo linhas duplicadas e tratando valores ausentes.
Transforme o valor da venda de uma moeda fictícia para USD usando a taxa de conversão de 1 FICT = 0.75 USD.
Carregue os dados limpos e transformados em um banco de dados relacional.

In [141]:

# Implementação estrutural

# Passo 1: Extração e Limpeza dos Dados
# Carrega o arquivo CSV
df_sales = pd.read_csv('input/sales_data.csv')

# Remove linhas duplicadas
df_sales = df_sales.drop_duplicates()

# Tratar valores ausentes (caso existam)
# Substitui valores ausentes por zero.
df_sales = df_sales.fillna(0)

# Passo 2: Transformação dos Dados
# Converter o valor da venda de moeda fictícia (FICT) para USD usando a taxa de conversão de 1 FICT = 0.75 USD
df_sales['usd_sale_value'] = df_sales['sale_value'] * 0.75

df_sales.to_csv("output/struct_sales_data.csv", index=False)

### 2. Análise com Apache Spark(utilize PySpark ou Spark)

#### Descrição

Dado um conjunto fictício de logs `website_logs.csv`:

- Identifique as 10 páginas mais visitadas.
- Calcule a média de duração das sessões dos usuários.
- Determine quantos usuários retornam ao site mais de uma vez por semana.

In [142]:
# Cria sessão Spark

spark = (SparkSession.builder
                     .master("local[1]")
                     .getOrCreate()
         )

In [143]:
# Importa o Dataframe
schema = StructType(
    [
          StructField("user_id", StringType())
         ,StructField("page_url", StringType())
         ,StructField("session_duration", FloatType())
         ,StructField("date", DateType())
    ]
)

df_website_access = spark.read.csv("input/website_logs.csv", header=True, schema=schema)
df_website_access.show()
df_website_access.printSchema()

+-------+-----------------+----------------+----------+
|user_id|         page_url|session_duration|      date|
+-------+-----------------+----------------+----------+
|  10001|    homepage.html|            15.0|2023-07-25|
|  10002|product_page.html|           120.0|2023-07-25|
|  10003|    checkout.html|            45.0|2023-07-25|
|  10004|     contact.html|            20.0|2023-07-25|
|  10005|    homepage.html|            10.0|2023-07-25|
|  10006|product_page.html|            95.0|2023-07-25|
|  10007|        blog.html|           150.0|2023-07-26|
|  10008|    homepage.html|            25.0|2023-07-26|
|  10009|product_page.html|            85.0|2023-07-26|
|  10010|    checkout.html|            50.0|2023-07-26|
|  10011|         faq.html|            35.0|2023-07-27|
|  10012|    homepage.html|            12.0|2023-07-27|
|  10013|product_page.html|           110.0|2023-07-27|
|  10014|    checkout.html|            60.0|2023-07-27|
|  10015|        blog.html|           160.0|2023

In [144]:
# 2.1 - Identifique as 10 páginas mais visitadas.

df_website_access.groupby("page_url")  \
    .count()                 \
    .sort(f.col("count")
          .desc())              \
    .show(10)

+-----------------+-----+
|         page_url|count|
+-----------------+-----+
|    homepage.html|   51|
|product_page.html|   46|
|    checkout.html|   35|
|     contact.html|   19|
|        blog.html|   18|
|         faq.html|   14|
|    about_us.html|    1|
+-----------------+-----+



In [145]:
# 2.2 Calcule a média de duração das sessões dos usuários.

df_website_access.select(f.avg("session_duration").alias("Duração média das sessões")) \
    .show(10)

+-------------------------+
|Duração média das sessões|
+-------------------------+
|        60.84239130434783|
+-------------------------+



In [149]:
# 2.3 Determine quantos usuários retornam ao site mais de uma vez por semana.

# Cria uma coluna que identifica a semana e o ano
df_website_access = df_website_access.withColumn("year_week",f.concat_ws("-",f.year("date"),f.weekofyear("date")))

# Cria um Dataframe com os usuários que retornam com mais de 1 acesso semanal
df_weekly_website_access = df_website_access.groupBy("user_id","year_week").count().filter("count > 1")

df_returning_users = df_weekly_website_access.select("user_id").distinct().count()
output_message = f"{df_returning_users} usuários retornam ao site mais de uma vez por semana."
print(output_message)


21 usuários retornam ao site mais de uma vez por semana.
